In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import tensorflow as tf
import re
import string


In [36]:
!conda install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: --allow-change-held-packages


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')

In [3]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naga3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\naga3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\naga3\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\naga3\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
def preprocess_tweet(tweet):
  tweet = tweet.lower() # lowercase

  tweet = re.sub(r"http\S+|https\S+","", tweet, flags=re.MULTILINE) #rem url

  tweet = tweet.translate(str.maketrans("", "", string.punctuation)) #punctuation

  tweet = re.sub(r"\@\w+|\#", "", tweet) # user ref and #

  #stop_Wrd
  tweet_tokens = word_tokenize(tweet)
  filtered_words = [word for word in tweet_tokens if word not in stop_words]
  

#stem
  ps = PorterStemmer()
  stemmed_words = [ps.stem(w) for w in filtered_words]

  #lemma
  lemmatizer = WordNetLemmatizer()
  lemma_words = [lemmatizer.lemmatize(w,pos='a') for w in stemmed_words] 

  return " ".join(lemma_words)


In [7]:
train_140 = pd.read_csv("training.csv",
                        encoding="latin-1",
                        names=['sentiment', 'num', "date", 'query','autor','text'])
train_140

,sentiment,num,date,query,autor,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [8]:
test_140 = pd.read_csv("testdata.csv",
                        encoding="latin-1",
                        names=['sentiment', 'num', "date", 'query','autor','text'])
test_140

,sentiment,num,date,query,autor,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [9]:
train_140 = train_140[['text','sentiment']]
train_140


,text,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0
...,...,...
1599995,Just woke up. Having no school is the best fee...,4
1599996,TheWDB.com - Very cool to hear old Walt interv...,4
1599997,Are you ready for your MoJo Makeover? Ask me f...,4
1599998,Happy 38th Birthday to my boo of alll time!!! ...,4


In [10]:
test_140 = test_140[['text','sentiment']]
test_140


,text,sentiment
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...,4
1,Reading my kindle2... Love it... Lee childs i...,4
2,"Ok, first assesment of the #kindle2 ...it fuck...",4
3,@kenburbary You'll love your Kindle2. I've had...,4
4,@mikefish Fair enough. But i have the Kindle2...,4
...,...,...
493,Ask Programming: LaTeX or InDesign?: submitted...,2
494,"On that note, I hate Word. I hate Pages. I hat...",0
495,Ahhh... back in a *real* text editing environm...,4
496,"Trouble in Iran, I see. Hmm. Iran. Iran so far...",0


In [11]:
data = [preprocess_tweet(t) for t in train_140['text']]


In [12]:
data_test = [preprocess_tweet(t) for t in test_140['text']]

In [13]:
(data_test)

['stellargirl loooooooovvvvvvee kindle2 dx cool 2 fantast right',
 'read kindle2 love lee child good read',
 'ok first asses kindle2 fuck rock',
 'kenburbari youll love kindle2 ive mine month never look back new big one huge need remors',
 'mikefish fair enough kindle2 think perfect',
 'richardebak big im quit happi kindle2',
 'fuck economi hate aig non loan given ass',
 'jqueri new best friend',
 'love twitter',
 'love obama make joke',
 'check video presid obama white hous correspond dinner',
 'karoli firmli believ obamapelosi zero desir civil charad slogan want destroy conservat',
 'hous correspond dinner last night whoopi barbara amp sherri went obama got stand ovat',
 'watchin espnju seen new nike commer puppet lebronsht hilariouslmao',
 'dear nike stop flywir shit wast scienc ugli love vincentx24x',
 'lebron best athlet gener time basketbal relat dont want get intersport debat 12',
 'talk guy last night tell die hard spur fan also told hate lebron jame',
 'love lebron',
 'ludajui

In [14]:
data_pd = pd.DataFrame(data)
data_test_pd = pd.DataFrame(data_test)

In [15]:
tokenizer = Tokenizer()

In [16]:
tokenizer.fit_on_texts(data_pd[0])

In [17]:
tokenizer.word_index

{'im': 1,
 'go': 2,
 'good': 3,
 'get': 4,
 'day': 5,
 'work': 6,
 'like': 7,
 'love': 8,
 'got': 9,
 'dont': 10,
 'today': 11,
 'time': 12,
 'cant': 13,
 'thank': 14,
 'want': 15,
 'back': 16,
 'miss': 17,
 'one': 18,
 'lol': 19,
 'know': 20,
 'u': 21,
 'see': 22,
 'feel': 23,
 'think': 24,
 'realli': 25,
 'well': 26,
 'hope': 27,
 'night': 28,
 'watch': 29,
 'need': 30,
 'still': 31,
 'make': 32,
 'new': 33,
 'na': 34,
 'amp': 35,
 'home': 36,
 'look': 37,
 'come': 38,
 'oh': 39,
 '2': 40,
 'much': 41,
 'last': 42,
 'twitter': 43,
 'morn': 44,
 'great': 45,
 'tomorrow': 46,
 'wish': 47,
 'wait': 48,
 'ill': 49,
 'sleep': 50,
 'that': 51,
 'haha': 52,
 'bad': 53,
 'sad': 54,
 'fun': 55,
 'tri': 56,
 'right': 57,
 'week': 58,
 'follow': 59,
 'happi': 60,
 'didnt': 61,
 'would': 62,
 'thing': 63,
 'sorri': 64,
 'tonight': 65,
 'say': 66,
 'friend': 67,
 'way': 68,
 'take': 69,
 'nice': 70,
 'gon': 71,
 'though': 72,
 'ive': 73,
 'hate': 74,
 'even': 75,
 'late': 76,
 'start': 77,
 'bed'

In [18]:
sequence = tokenizer.texts_to_sequences(data_pd[0])


In [19]:
sequence_test = tokenizer.texts_to_sequences(data_test_pd[0])

In [20]:
tokenizer.word_index['love']

8

In [21]:
sequence_test

[[315772, 99758, 8006, 120, 40, 886, 57],
 [140, 99758, 8, 2439, 1549, 3, 140],
 [114, 105, 18210, 99758, 211, 350],
 [698789,
  454,
  8,
  99758,
  73,
  297,
  281,
  98,
  37,
  16,
  33,
  154,
  18,
  649,
  30,
  16066],
 [658, 310, 99758, 24, 533],
 [600493, 154, 1, 360, 60, 99758],
 [211, 3751, 74, 70200, 2495, 4157, 1458, 452],
 [6910, 33, 106, 67],
 [8, 43],
 [8, 2649, 32, 850],
 [177, 235, 3591, 2649, 700, 150, 13236, 285],
 [34631, 16820, 269, 2783, 4196, 6127, 17842, 13313, 15, 2473, 401088],
 [150, 13236, 285, 42, 28, 17753, 7815, 35, 8780, 146, 2649, 9, 800, 26630],
 [957, 343, 33, 5073, 17824, 6291],
 [504, 5073, 194, 204447, 290, 913, 1300, 1411, 8, 293931],
 [2868, 106, 5675, 1144, 12, 2204, 1705, 10, 15, 4, 2833, 535],
 [126, 89, 42, 28, 160, 242, 186, 7097, 325, 192, 432, 74, 2868, 1481],
 [8, 2868],
 [12328, 2868, 3409, 1, 31, 601, 111, 140988, 183],
 [2868, 1270],
 [2868, 4256, 1264, 19, 8, 806, 97, 2, 1807, 19],
 [2868, 95, 9534],
 [92513, 2868, 3409, 950, 2689,

In [22]:
model = tf.keras.Sequential()

In [23]:
X_train = pad_sequences(sequence)
len_text = len(X_train[999])
len_text

56

In [24]:
X_test = pad_sequences(sequence_test,maxlen = len_text)
text = len(X_test[0])
text

56

In [25]:
model = Sequential()
model.add(layers.Embedding(1000,64))
model.add(layers.LSTM(128))
model.add(layers.Dense(1,activation='sigmoid'))

In [26]:

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model_save_path = "best_model.h5"
check_p = ModelCheckpoint(model_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [28]:
Y_train = train_140[train_140.sentiment == 0].sentiment
Y_train2 = train_140[train_140.sentiment == 4].sentiment - 3
Y_train = Y_train.append(Y_train2)
Y_train

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: sentiment, Length: 1600000, dtype: int64

In [29]:
X_train

array([[     0,      0,      0, ...,   7673,   1676,      5],
       [     0,      0,      0, ...,     11,    192,   1075],
       [     0,      0,      0, ...,    355,      2,   2838],
       ...,
       [     0,      0,      0, ...,   6417,    296,   1545],
       [     0,      0,      0, ...,  10438,  90209,  71955],
       [     0,      0,      0, ..., 225968, 725606, 725607]])

In [33]:
history = model.fit(X_train,
                    Y_train,
                    epochs= 5,
                    batch_size = 128,
                    validation_split=0.1)
                    #callbacks=[check_p])

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding/embedding_lookup' defined at (most recent call last):
    File "C:\Anaconda\envs\TensorFlowenv\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Anaconda\envs\TensorFlowenv\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Anaconda\envs\TensorFlowenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Anaconda\envs\TensorFlowenv\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Anaconda\envs\TensorFlowenv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Anaconda\envs\TensorFlowenv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Anaconda\envs\TensorFlowenv\lib\asyncio\base_eve